In [16]:
# coding=utf-8
"""
This module contains the tests which ensure that the flashloan tokens click parameters are respected.
"""
from fastlane_bot import Bot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3,  CarbonV1, BancorV3
import subprocess, os, sys
import pytest
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v3.3.1 (05/Oct/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.2 (2023-08-27)
UniswapV3 v0.0.2 (2023-08-27)
CarbonV1 v0.0.2 (2023-08-27)
BancorV3 v0.0.2 (2023-08-27)
Version = 3-b2.2 [requirements >= 3.0 is met]


# Setup

In [17]:
def find_main_py():
    # Start at the directory of the current script
    cwd = os.path.abspath(os.path.join(os.getcwd()))
    
    print(f"Searching for main.py in {cwd}")
    while True:
        # Check if main.py exists in the current directory
        if "main.py" in os.listdir(cwd):
            return cwd  # Found the directory containing main.py
        else:
            # If not, go up one directory
            new_cwd = os.path.dirname(cwd)

            # If we're already at the root directory, stop searching
            if new_cwd == cwd:
                raise FileNotFoundError("Could not find main.py in any parent directory")

            cwd = new_cwd
       
       
def run_command(arb_mode, expected_log_line):
    
    # Find the correct path to main.py
    main_script_path = find_main_py()
    print(f"Found main.py in {main_script_path}")
    main_script_path = os.path.normpath(main_script_path + "/main.py")

    # Run the command
    cmd = [
        "python",
        main_script_path,
        f"--arb_mode={arb_mode}",
        "--default_min_profit_gas_token=0.001",
        "--limit_bancor3_flashloan_tokens=False",
        "--use_cached_events=False",
        "--timeout=1",
        "--loglevel=DEBUG",
        "--flashloan_tokens='0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C,0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE,0xAa6E8127831c9DE45ae56bB1b0d4D4Da6e5665BD'",
        "--blockchain=ethereum"
    ]
    subprocess.Popen(cmd)
        
    # Wait for the expected log line to appear
    found = False
    result = subprocess.run(cmd, text=True, capture_output=True, check=True, timeout=20)

    # Check if the expected log line is in the output
    if expected_log_line in result.stderr or expected_log_line in result.stdout:
        found = True

    if not found:
        pytest.fail("Expected log line was not found within 1 minute")  # If we reach this point, the test has failed

## Test flashloan_tokens is Respected

In [18]:
expected_log_line = """Flashloan tokens are set as: ["'0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C", 'ETH', "0xAa6E8127831c9DE45ae56bB1b0d4D4Da6e5665BD'"]"""
arb_mode = "multi"
run_command(arb_mode=arb_mode, expected_log_line=expected_log_line)

Searching for main.py in /Users/mikewcasale/Documents/GitHub/bancor/fastlane-bot/resources/NBTest
Found main.py in /Users/mikewcasale/Documents/GitHub/bancor/fastlane-bot


2023-12-19 03:52:37,861 [fastlane:INFO] - 
2023-12-19 03:52:37,861 [fastlane:INFO] - **********************************************
2023-12-19 03:52:37,861 [fastlane:INFO] - The logging path is set to: logs/20231219-035237/bot.log
2023-12-19 03:52:37,861 [fastlane:INFO] - **********************************************
2023-12-19 03:52:37,861 [fastlane:INFO] - 
2023-12-19 03:52:38,662 [fastlane:INFO] - [events.utils.get_config] Using mainnet config
2023-12-19 03:52:38,676 [fastlane:INFO] - tokens: 23041, 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
2023-12-19 03:52:38,682 [fastlane:INFO] - unique_tokens: 23041
2023-12-19 03:52:38,682 [fastlane:INFO] - [events.utils.handle_flashloan_tokens] unique_tokens: 23041
2023-12-19 03:52:38,689 [fastlane:INFO] - [events.utils.handle_flashloan_tokens] Flashloan tokens are set as: ["'0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C", 'ETH', "0xAa6E8127831c9DE45ae56bB1b0d4D4Da6e5665BD'"], <class 'list'>
2023-12-19 03:52:38,689 [fastlane:INFO] - [events.utils